In [1]:
import unittest
import pandas as pd
import tushare as QATs
from QUANTAXIS import QUANTAXIS as QA
from QUANTAXIS.QAARP.QARisk import QA_Performance, QA_Risk
from QUANTAXIS.QAARP.QAStrategy import QA_Strategy
from QUANTAXIS.QAARP.QAUser import QA_User
from QUANTAXIS.QABacktest.QABacktest import QA_Backtest
from QUANTAXIS.QAFetch.QATdx import QA_fetch_get_stock_day
from QUANTAXIS.QAIndicator import QA_indicator_MA
from QUANTAXIS.QAUtil.QADate import QA_util_datetime_to_strdate
from QUANTAXIS.QAUtil.QALogs import QA_util_log_info
from QUANTAXIS.QAUtil.QAParameter import (AMOUNT_MODEL, DATASOURCE, FREQUENCE,
                                          MARKET_TYPE, ORDER_DIRECTION,
                                          ORDER_MODEL, OUTPUT_FORMAT)

In [2]:
class MAStrategy(QA_Strategy):
    def __init__(self):
        super().__init__()
        self.frequence = FREQUENCE.DAY					        #日线
        self.market_type = MARKET_TYPE.STOCK_CN			        #市场类型：中国stock
        self.date_start = '2016-01-01' 		                    #获取数据的开始时间
        self.date_end = '2018-01-01'		                    #获取数据的回测结束时间
        self.df_from_Tdx = QA_fetch_get_stock_day(
            '300439', self.date_start, self.date_end, '01')		#数据获取TDX
        '''--------------初始化技术指标----------------'''
        self.ma05 = QA_indicator_MA(self.df_from_Tdx, 5)		#5日均线
        self.ma10 = QA_indicator_MA(self.df_from_Tdx, 10)		#10日均线
        self.ma15 = QA_indicator_MA(self.df_from_Tdx, 15)		#15日均线
        self.ma20 = QA_indicator_MA(self.df_from_Tdx, 20)		#20日均线
    def on_bar(self, event):
        sellavailable = self.sell_available
        for item in event.market_data.code:
            current_date = self.current_time.date()
            today_T = pd.Timestamp(current_date)
            vma05 = self.ma05.at[today_T, 'MA']
            vma10 = self.ma10.at[today_T, 'MA']
            vma15 = self.ma15.at[today_T, 'MA']
            vma20 = self.ma20.at[today_T, 'MA']
            if vma05 > vma10 and vma10 > vma15 and vma15 > vma20:
                if self.sell_available is not None and self.sell_available.get(item, 0) == 0:
                    print("均线多头排列，买入 buy %d" % (100))
                    print(event.send_order)
                    print(type(event.send_order))
                    event.send_order(account_id=self.account_cookie,
                                     amount=100,#订单一手
                                     amount_model=AMOUNT_MODEL.BY_AMOUNT,#订单计算模式
                                     time=self.current_time,
                                     code=item,
                                     price=0,
                                     order_model=ORDER_MODEL.MARKET,
                                     towards=ORDER_DIRECTION.BUY,#订单类型：买单
                                     market_type=self.market_type,
                                     frequence=self.frequence,
                                     broker_name=self.broker)
            elif vma05 < vma10 and vma10 < vma15 and vma15 < vma20:
                if self.sell_available is not None and self.sell_available.get(item, 0) > 0:
                    print("均线空头排列，卖出%d" % 100)
                    event.send_order(account_id=self.account_cookie,
                                     amount=self.sell_available[item],
                                     amount_model=AMOUNT_MODEL.BY_AMOUNT,
                                     time=self.current_time,
                                     code=item,
                                     price=0,
                                     order_model=ORDER_MODEL.MARKET,
                                     towards=ORDER_DIRECTION.SELL,#订单类型：卖单
                                     market_type=self.market_type,
                                     frequence=self.frequence,
                                     broker_name=self.broker)
            else:
                pass
class Backtest(QA_Backtest):
    def __init__(self, market_type, frequence, start, end, code_list, commission_fee):
        super().__init__(market_type,  frequence, start, end, code_list, commission_fee)
        self.user = QA_User()
        mastrategy = MAStrategy()
        self.portfolio, self.account = self.user.register_account(mastrategy)
    def after_success(self):
        QA_util_log_info(self.account.history_table)
        # check if the history_table is empty list
        if len(self.account.history_table) == 0:
            # 没有交易历史记录，直接返回
            return

        risk = QA_Risk(self.account, benchmark_code='000300',  #沪深300
                       benchmark_type=MARKET_TYPE.INDEX_CN)
        print(risk().T)
        risk.plot_assets_curve()
        #risk.plot_dailyhold()
        #risk.plot_signal()
        #performance = QA_Performance(self.account)
        #performance.plot_pnlmoney(performance.pnl_fifo)
        #performance.plot_pnlratio(performance.pnl_fifo)
        self.account.save()
        risk.save()
class Test_QABacktest(unittest.TestCase):
    def testBacktraceTest(self):#这个是否可以直接删除该方法
        self.run_daybacktest()
        pass
    def run_daybacktest(self):
        self.time_to_Market_300439 = '2016-01-01'           #回测开始时间
        self.time_to_day = '2018-01-01'                     #回测结束时间
        backtest = Backtest(market_type=MARKET_TYPE.STOCK_CN,
                            frequence=FREQUENCE.DAY,
                            start=self.time_to_Market_300439,
                            end=self.time_to_day,
                            code_list=['300439'],
                            commission_fee=0.00015)
        backtest.start_market()
        backtest.run()
        backtest.stop()
        print("结束回测！")


if __name__ == '__main__':
    Test_QABacktest().testBacktraceTest()#该处是否可以直接修改为 Test_QABacktest().run_daybacktest()

QUANTAXIS>> Selecting the Best Server IP of TDX


USING DEFAULT STOCK IP
️Bad FUTUREIP REPSONSE 124.74.236.94
DEFAULT FUTURE IP is BAD, RETESTING
️Bad FUTUREIP REPSONSE 124.74.236.94
️Bad FUTUREIP REPSONSE 218.80.248.229
️Bad FUTUREIP REPSONSE 124.74.236.94
️Bad FUTUREIP REPSONSE 58.246.109.27
️Bad FUTUREIP REPSONSE 112.74.214.43
️Bad FUTUREIP REPSONSE 120.24.0.77
️Bad FUTUREIP REPSONSE 106.14.95.149
️Bad FUTUREIP REPSONSE 119.97.185.5
️Bad FUTUREIP REPSONSE 202.103.36.71
️Bad FUTUREIP REPSONSE 59.175.238.38
️Bad FUTUREIP REPSONSE 113.105.142.136
BAD RESPONSE 61.152.107.141
BAD RESPONSE 61.152.107.171
BAD RESPONSE 119.147.86.171
️Bad FUTUREIP REPSONSE 47.92.127.181


QUANTAXIS>> === The BEST SERVER ===
 stock_ip 114.80.80.222 future_ip 124.74.236.94


'open'


TypeError: 'NoneType' object is not subscriptable